<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ca47168f092af690fb4004b32fd5b984f8620b857502cce714e56accc720637c
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-03 14:59:50
-------------------
qualified stocks: 90
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  5.00 L
Current:  1.40 C
-------------------
Today PnL: 58.98 K (0.42%)
Current PnL: -17.90 L (-12.04%)
CY Booked + Current PnL: -8.33 L (-5.6%)
-------------------
Total profit:  3.02 L
Total loss:  -20.92 L
-------------------
Total Booked + Current PnL: 18.65 L (15.33%)
Total Booked PnL: 36.55 L (30.03%)
Curr Year Booked PnL: 9.57 L (6.84%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.94 L (56.44%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DIXON,18940.15,-29.05,69.0,H-MC,6.83,141920.0,26984.0,9608.0,1.07,23.48,6.77,31.83,81.0,2.81,1.05,47.10,X40N,ATH,IT
80,TTKPRESTIG,770.00,107.26,71.0,M-SC,6.97,89322.0,-11455.0,11549.0,-0.02,-11.37,12.93,0.09,245.0,-0.99,0.66,17.35,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-14.86,57.0,H-LC,8.93,170027.0,13850.0,12242.0,0.61,8.87,7.20,16.71,22.0,1.13,1.26,27.85,X40N,BTT,HEALTHCARE
65,SBIN,863.00,-13.25,50.0,M-LC,4.45,215087.0,13608.0,13615.0,0.96,6.75,6.33,13.51,60.0,1.00,1.59,20.20,XY25,NTT,BANKS
15,BRITANNIA,6362.67,17.62,65.0,H-LC,9.15,212796.0,14700.0,16258.0,0.33,7.42,7.64,15.63,26.0,0.90,1.58,30.89,XY25,ATH,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,41.14,61.0,M-MC,7.60,222844.0,-2118.0,169763.0,5.13,-0.94,76.18,74.53,192.0,-0.01,1.65,29.38,XY24,BTT,STEEL
48,KPIGREEN,731.35,5.18,44.0,H-SC,11.08,125118.0,-179.0,59181.0,0.90,-0.14,47.30,47.09,141.0,-0.00,0.93,56.59,MH,ATH,POWER
54,PGHH,18062.58,-30.28,55.0,H-MC,5.37,201945.0,1125.0,68984.0,-1.04,0.56,34.16,34.92,80.0,0.02,1.50,5.26,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,1943.91,0.65,41.0,H-LC,9.92,229691.0,-12233.0,75499.0,-0.18,-5.06,32.87,26.15,8.0,-0.16,1.70,8.54,X40,ATH,IT
46,JIOFIN,387.00,-1.31,41.0,H-LC,14.28,218330.0,924.0,52574.0,0.05,0.43,24.08,24.61,48.0,0.02,1.62,55.48,XY24,BTT,FINANCE
59,RELIANCE,1533.00,-14.79,42.0,H-LC,7.65,213564.0,3198.0,25585.0,0.18,1.52,11.98,13.68,37.0,0.12,1.58,18.29,XY25,NTT,REFINERIES
42,INFY,2275.00,-19.24,46.0,H-LC,11.29,314708.0,1820.0,169879.0,-1.41,0.58,53.98,54.87,3.0,0.01,2.33,7.27,X40,BTT,IT
77,TCS,4998.00,-27.08,47.0,H-LC,14.72,290902.0,-55054.0,178905.0,-0.54,-15.91,61.50,35.80,1.0,-0.31,2.16,3.05,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7230.20,-6.36,50.0,H-LC,2.36,232538.0,-13613.0,92829.0,-1.45,-5.53,39.92,32.18,16.0,-0.15,1.72,28.52,X200,ATH,IT
53,NESTLEIND,1377.00,-9.37,59.0,H-LC,3.09,278811.0,13385.0,43411.0,-0.78,5.04,15.57,21.40,11.0,0.31,2.07,13.22,XY25,NTT,FMCG
44,ITC,452.00,-37.92,53.0,H-LC,3.26,201150.0,1012.0,19874.0,1.13,0.51,9.88,10.44,4.0,0.05,1.49,6.10,X40,NTT,FMCG
20,CIPLA,1795.00,-19.09,57.0,H-LC,5.08,215953.0,11453.0,29953.0,0.45,5.60,13.87,20.25,10.0,0.38,1.60,14.82,X40N,BTT,PHARMA
86,VBL,671.64,-10.56,53.0,H-LC,7.46,322190.0,6348.0,106323.0,1.16,2.01,33.00,35.67,5.0,0.06,2.39,16.14,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,41.14,61.0,M-MC,7.60,222844.0,-2118.0,169763.0,5.13,-0.94,76.18,74.53,192.0,-0.01,1.65,29.38,XY24,BTT,STEEL
8,AWL,485.00,-64.07,61.0,H-MC,6.29,220646.0,-47801.0,179473.0,3.90,-17.81,81.34,49.05,116.0,-0.27,1.64,13.45,XY24,NTT,FMCG
52,NATIONALUM,247.44,-45.32,71.0,H-MC,0.48,109583.0,9079.0,21555.0,3.11,9.03,19.67,30.49,79.0,0.42,0.81,44.37,MH,ATH,METALS
58,RELAXO,1176.00,-30.87,64.0,H-SC,6.29,64192.0,-51346.0,86338.0,3.06,-44.44,134.50,30.28,136.0,-0.59,0.48,26.20,X40N,NTT,FOOTWEAR
71,STARHEALTH,761.00,10.73,59.0,H-SC,2.96,164403.0,-35813.0,111843.0,2.99,-17.89,68.03,37.97,144.0,-0.32,1.22,32.00,XY24,NTT,INSURANCE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HONAUT,58357.33,-21.77,39.0,H-SC,3.61,110190.0,-17748.0,64880.0,-2.47,-13.87,58.88,36.84,143.0,-0.27,0.82,12.81,X40N,ATH,ELECTRICAL
66,SFL,1287.00,42.17,61.0,M-SC,8.95,187785.0,-74433.0,144256.0,-1.56,-28.39,76.82,26.63,229.0,-0.52,1.39,16.75,XY24,NTT,MISC
50,LTIM,7230.20,-6.36,50.0,H-LC,2.36,232538.0,-13613.0,92829.0,-1.45,-5.53,39.92,32.18,16.0,-0.15,1.72,28.52,X200,ATH,IT
42,INFY,2275.00,-19.24,46.0,H-LC,11.29,314708.0,1820.0,169879.0,-1.41,0.58,53.98,54.87,3.0,0.01,2.33,7.27,X40,BTT,IT
23,COLPAL,3767.14,2.04,60.0,H-MC,9.58,185905.0,-25836.0,107936.0,-1.26,-12.20,58.06,38.77,84.0,-0.24,1.38,10.65,XY25,ATH,FMCG


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TTKPRESTIG,770.0,107.26,71.0,M-SC,6.97,89322.0,-11455.0,11549.0,-0.02,-11.37,12.93,0.09,245.0,-0.99,0.66,17.35,OX40N,NTT,DURABLES
81,UJJIVANSFB,60.0,84.36,45.0,H-SC,13.82,116505.0,-25974.0,45495.0,0.19,-18.23,39.05,13.70,163.0,-0.57,0.86,37.51,OX40N,NTT,BANKS
40,INDIGOPNTS,1408.0,116.52,52.0,M-SC,3.04,141881.0,-32678.0,32718.0,2.10,-18.72,23.06,0.02,221.0,-1.00,1.05,23.41,OX40N,NTT,PAINTS
47,KANSAINER,340.0,-68.34,54.0,H-SC,3.71,218817.0,-50850.0,87177.0,0.47,-18.86,39.84,13.47,138.0,-0.58,1.62,10.67,XY24,NTT,PAINTS
19,CERA,9475.0,-20.29,43.0,H-SC,3.09,113580.0,-30223.0,56972.0,-0.36,-21.02,50.16,18.60,149.0,-0.53,0.84,25.34,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,4998.00,-27.08,47.0,H-LC,14.72,290902.0,-55054.0,178905.0,-0.54,-15.91,61.50,35.80,1.0,-0.31,2.16,3.05,X40,BTT,IT
42,INFY,2275.00,-19.24,46.0,H-LC,11.29,314708.0,1820.0,169879.0,-1.41,0.58,53.98,54.87,3.0,0.01,2.33,7.27,X40,BTT,IT
44,ITC,452.00,-37.92,53.0,H-LC,3.26,201150.0,1012.0,19874.0,1.13,0.51,9.88,10.44,4.0,0.05,1.49,6.10,X40,NTT,FMCG
86,VBL,671.64,-10.56,53.0,H-LC,7.46,322190.0,6348.0,106323.0,1.16,2.01,33.00,35.67,5.0,0.06,2.39,16.14,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,49.0,H-LC,13.42,248789.0,-12830.0,132032.0,0.68,-4.90,53.07,45.57,7.0,-0.10,1.84,5.86,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.44,186935.0,36022.0,21273.0,0.55,23.87,11.38,37.97,93.0,1.69,1.39,55.21,AR,ATH,AUTO
26,DIXON,18940.15,-29.05,69.0,H-MC,6.83,141920.0,26984.0,9608.0,1.07,23.48,6.77,31.83,81.0,2.81,1.05,47.10,X40N,ATH,IT
27,DMART,5201.00,-7.03,69.0,H-LC,14.35,232108.0,47627.0,22747.0,1.22,25.82,9.80,38.14,38.0,2.09,1.72,39.16,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,12.39,64.0,H-MC,3.42,194030.0,29360.0,38612.0,-0.41,17.83,19.90,41.28,89.0,0.76,1.44,27.24,X40N,ATH,AC
87,VOLTAS,1918.49,1.72,65.0,H-MC,4.98,213390.0,21648.0,74388.0,-1.25,11.29,34.86,50.08,99.0,0.29,1.58,18.86,XY25,ATH,AC
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.44,186935.0,36022.0,21273.0,0.55,23.87,11.38,37.97,93.0,1.69,1.39,55.21,AR,ATH,AUTO
24,DABUR,735.00,-2.58,64.0,H-MC,3.94,211926.0,14898.0,74725.0,-0.22,7.56,35.26,45.49,102.0,0.20,1.57,21.59,XY24,BTT,FMCG
18,CAMS,5211.76,-6.49,47.0,H-SC,5.62,106372.0,4368.0,39560.0,-0.08,4.28,37.19,43.06,122.0,0.11,0.79,22.13,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.44,186935.0,36022.0,21273.0,0.55,23.87,11.38,37.97,93.0,1.69,1.39,55.21,AR,ATH,AUTO
52,NATIONALUM,247.44,-45.32,71.0,H-MC,0.48,109583.0,9079.0,21555.0,3.11,9.03,19.67,30.49,79.0,0.42,0.81,44.37,MH,ATH,METALS
88,WHIRLPOOL,2270.00,-40.29,58.0,M-SC,4.91,100830.0,9332.0,69421.0,0.80,10.20,68.85,86.07,223.0,0.13,0.75,46.42,XR,NTT,DURABLES
39,INDIAMART,4850.92,-52.03,43.0,H-SC,12.49,134037.0,10701.0,123059.0,-0.75,8.68,91.81,108.45,119.0,0.09,0.99,32.97,AR,ATH,MISC
89,WIPRO,420.00,-11.98,50.0,M-LC,7.07,154839.0,3894.0,105554.0,-0.47,2.58,68.17,72.51,53.0,0.04,1.15,8.76,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SIS,528.00,2244.04,55.0,H-SC,5.74,63070.0,-18020.0,26691.0,1.42,-22.22,42.32,10.69,156.0,-0.68,0.47,27.82,OX40N,NTT,MISC
63,SAMMAANCAP,326.00,-172.73,68.0,M-SC,7.28,83238.0,-18972.0,112363.0,-0.45,-18.56,134.99,91.37,208.0,-0.17,0.62,35.12,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-2.66,49.0,H-SC,16.23,90666.0,-10297.0,104828.0,2.27,-10.20,115.62,93.63,148.0,-0.10,0.67,35.04,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,33.33,92352.0,-21197.0,61248.0,0.88,-18.67,66.32,35.27,268.0,-0.35,0.68,126.03,XR,NTT,HOTELS
51,MASFIN,398.61,-17.80,53.0,H-SC,13.25,93615.0,-4365.0,25969.0,1.66,-4.45,27.74,22.05,152.0,-0.17,0.69,36.08,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,33.33,92352.0,-21197.0,61248.0,0.88,-18.67,66.32,35.27,268.0,-0.35,0.68,126.03,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,0.81,78.0,H-MC,19.44,186935.0,36022.0,21273.0,0.55,23.87,11.38,37.97,93.0,1.69,1.39,55.21,AR,ATH,AUTO
58,RELAXO,1176.00,-30.87,64.0,H-SC,6.29,64192.0,-51346.0,86338.0,3.06,-44.44,134.50,30.28,136.0,-0.59,0.48,26.20,X40N,NTT,FOOTWEAR
27,DMART,5201.00,-7.03,69.0,H-LC,14.35,232108.0,47627.0,22747.0,1.22,25.82,9.80,38.14,38.0,2.09,1.72,39.16,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,12.39,64.0,H-MC,3.42,194030.0,29360.0,38612.0,-0.41,17.83,19.90,41.28,89.0,0.76,1.44,27.24,X40N,ATH,AC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.60
1,25,43.36
2,50,73.41


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    37.43
SC    37.18
MC    25.39
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.21
X40      15.39
X40N     14.25
XY25     11.47
AR        9.36
XR        8.97
OX40N     7.36
MH        1.74
X200      1.72
X5K       1.44
SR        1.09
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.24
H-MC    22.42
H-SC    22.27
M-SC    13.32
M-LC     7.12
M-MC     2.65
L-SC     1.59
L-LC     1.07
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.34,1.23,30.94
IT,13.26,-15.41,72.65
FINANCE,7.31,-22.44,67.58
BANKS,7.27,-15.94,65.17
PAINTS,5.97,-14.67,40.34
MISC,5.81,-17.21,84.40
ELECTRICAL,5.66,-7.83,67.25
AUTO,4.59,-9.14,56.61
AC,3.75,4.16,28.29


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2794180.0,22
AR,1217228.0,10
XR,1083529.0,13
X40,892621.0,10
X40N,513435.0,11
OX40N,464896.0,10
XY25,433951.0,8
SR,196691.0,2
X5K,123723.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2773538.0,27
M-SC,1448428.0,17
H-LC,1404927.0,17
H-MC,1183358.0,17
M-LC,417574.0,5
M-MC,320467.0,2
L-SC,246273.0,3
L-MC,58859.0,1
L-LC,40395.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,930970.0,6
M-SC,XY24,861276.0,7
H-SC,AR,666507.0,5
H-LC,X40,593562.0,5
H-SC,XR,525228.0,6
H-MC,XY24,454211.0,4
H-LC,AR,370020.0,2
M-MC,XY24,320467.0,2
H-MC,X40,221492.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
